In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score


In [2]:
def load_images_from_folder(folder, label, img_size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            images.append(img)
            labels.append(label)
    return images, labels

def load_dataset(data_dir):
    cat_images, cat_labels = load_images_from_folder(os.path.join(data_dir, 'cat'), label=0)
    dog_images, dog_labels = load_images_from_folder(os.path.join(data_dir, 'dog'), label=1)

    images = cat_images + dog_images
    labels = cat_labels + dog_labels

    return np.array(images), np.array(labels)

train_images, train_labels = load_dataset('./dogs-vs-cats/train')
test_images, test_labels = load_dataset('./dogs-vs-cats/test1')

# Normalize the images
train_images = train_images / 255.0
test_images = test_images / 255.0

# Flatten the images for the SVM
train_images_flattened = train_images.reshape(len(train_images), -1)
test_images_flattened = test_images.reshape(len(test_images), -1)


FileNotFoundError: [WinError 3] The system cannot find the path specified: './dogs-vs-cats/train\\cat'